# Overview

- Defining the problem of persistent data
- Key concepts with containers: immutable, ephemeral
- Learning and using Data Volumes
- Learning and using Bind Mounts

## Container Lifetime and Pesistent Data
- Containers are usually immutable and ephemeral
- "Immutable Infrastructure": Only re-deploy containers, never change
- This is the ideal scenario, but what about databases or unique data
- Docker gives us features to ensure these "Separation of Concerns"
- This is known as "Persistent Data"
- Two ways: Volume and Bind Mounts
- Volumes: Make special location outside of container UFS
- Bind Mounts: link container path to host path

### Volumes

**Prune Volumes**  
- `>docker prune volumes`

In [3]:
!docker image inspect mysql

[
    {
        "Id": "sha256:db2b37ec6181ee1f367363432f841bf3819d4a9f61d26e42ac16e5bd7ff2ec18",
        "RepoTags": [
            "mysql:latest"
        ],
        "RepoDigests": [
            "mysql@sha256:8c17271df53ee3b843d6e16d46cff13f22c9c04d6982eb15a9a47bd5c9ac7e2d"
        ],
        "Parent": "",
        "Comment": "",
        "Created": "2020-10-22T22:28:20.110788809Z",
        "Container": "4cd656b8935175a8fa44d9ad18c4e33a272c4b88a6362d2680e724ae059a619e",
        "ContainerConfig": {
            "Hostname": "4cd656b89351",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "ExposedPorts": {
                "3306/tcp": {},
                "33060/tcp": {}
            },
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bi

In [4]:
!docker container run -d --name mysql -e MYSQL_ALLOW_EMPTY_PASSWORD=True mysql

224b5b064ec6ff2b52e29cba78b480cd010a646ffdacba95690a53671d4f9393


In [5]:
!docker container ls

CONTAINER ID        IMAGE                 COMMAND                  CREATED             STATUS              PORTS                  NAMES
224b5b064ec6        mysql                 "docker-entrypoint.s…"   9 seconds ago       Up 8 seconds        3306/tcp, 33060/tcp    mysql
498ec070b0e3        cvat/ui               "/docker-entrypoint.…"   7 days ago          Up 2 hours          80/tcp                 cvat_ui
9710bbd9e905        nginx:stable-alpine   "/docker-entrypoint.…"   7 days ago          Up 2 hours          0.0.0.0:8080->80/tcp   cvat_proxy
d9f68ac2b68e        cvat/server           "/usr/bin/supervisord"   7 days ago          Up 2 hours          8080/tcp, 8443/tcp     cvat
2b8a2aa219d5        redis:4.0-alpine      "docker-entrypoint.s…"   7 days ago          Up 2 hours          6379/tcp               cvat_redis
47c63fc28748        postgres:10-alpine    "docker-entrypoint.s…"   7 days ago          Up 2 hours          5432/tcp               cvat_db


In [6]:
!docker container inspect mysql

[
    {
        "Id": "224b5b064ec6ff2b52e29cba78b480cd010a646ffdacba95690a53671d4f9393",
        "Created": "2020-11-07T04:43:00.296528087Z",
        "Path": "docker-entrypoint.sh",
        "Args": [
            "mysqld"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 3159,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2020-11-07T04:43:00.737416733Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:db2b37ec6181ee1f367363432f841bf3819d4a9f61d26e42ac16e5bd7ff2ec18",
        "ResolvConfPath": "/var/lib/docker/containers/224b5b064ec6ff2b52e29cba78b480cd010a646ffdacba95690a53671d4f9393/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/224b5b064ec6ff2b52e29cba78b480cd010a646ffdacba95690a53671d4f9393/hostname",
        "HostsPa

In [7]:
!docker volume ls

DRIVER              VOLUME NAME
local               1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a
local               7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6
local               cvat_cvat_data
local               cvat_cvat_db
local               cvat_cvat_keys
local               cvat_cvat_logs
local               cvat_cvat_models


In [8]:
!docker volume inspect 1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a

[
    {
        "CreatedAt": "2020-11-07T04:40:35Z",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a/_data",
        "Name": "1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a",
        "Options": null,
        "Scope": "local"
    }
]


In [9]:
!docker volume inspect 7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6

[
    {
        "CreatedAt": "2020-11-07T04:43:16Z",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6/_data",
        "Name": "7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6",
        "Options": null,
        "Scope": "local"
    }
]


In [10]:
!docker container stop mysql
!docker container ls -a

mysql
CONTAINER ID        IMAGE                 COMMAND                  CREATED             STATUS                              PORTS                  NAMES
224b5b064ec6        mysql                 "docker-entrypoint.s…"   5 minutes ago       Exited (0) Less than a second ago                          mysql
498ec070b0e3        cvat/ui               "/docker-entrypoint.…"   7 days ago          Up 2 hours                          80/tcp                 cvat_ui
9710bbd9e905        nginx:stable-alpine   "/docker-entrypoint.…"   7 days ago          Up 2 hours                          0.0.0.0:8080->80/tcp   cvat_proxy
d9f68ac2b68e        cvat/server           "/usr/bin/supervisord"   7 days ago          Up 2 hours                          8080/tcp, 8443/tcp     cvat
2b8a2aa219d5        redis:4.0-alpine      "docker-entrypoint.s…"   7 days ago          Up 2 hours                          6379/tcp               cvat_redis
47c63fc28748        postgres:10-alpine    "docker-entrypoint.s…"   7 da

In [12]:
!docker volume ls

DRIVER              VOLUME NAME
local               1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a
local               7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6
local               cvat_cvat_data
local               cvat_cvat_db
local               cvat_cvat_keys
local               cvat_cvat_logs
local               cvat_cvat_models


In [13]:
!docker container rm mysql

mysql


### Named Volume

In [14]:
!docker container run -d --name mysql -e MYSQL_ALLOW_EMPTY_PASSWORD=True -v mysql-db:/var/lib/mysql mysql

9879bbfd4d11a72f31bfe663726055a68cc4bc752f6d5290d811b98b534a0ac4


In [16]:
!docker volume ls

DRIVER              VOLUME NAME
local               1db34a8dc27f864e89c217ffecb07ed2014a7c5b23d95f4a58f8e74fe5b3b60a
local               7515f955645cd93fc7ed5a87175ea4afd24341b57c12f214bd244a32f235a5e6
local               cvat_cvat_data
local               cvat_cvat_db
local               cvat_cvat_keys
local               cvat_cvat_logs
local               cvat_cvat_models
local               mysql-db
